In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import datetime

In [2]:
# headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
headers = {'User-Agent': 'Mozilla/3.0 (Win95; U)'}
raw_html = requests.get('https://old.reddit.com', headers=headers)
soup_doc = BeautifulSoup(raw_html.content, "html.parser")

In [3]:
entries = soup_doc.find_all(class_ = 'thing')
frontpage_posts = []
for entry in entries:
    try:
        post = {}
        post['title'] = entry.find('a', class_ = 'title').text
        post['author'] = entry.find(class_ = 'author').text
        post['author_profile'] = entry.find(class_ = 'author')['href']
        post['content_link'] = entry.find('a', class_ = 'title', href = True)['href']
        post['subreddit'] = entry.find(class_ = 'subreddit').text
        post['subreddit_link'] = entry.find(class_ = 'subreddit')['href']
        post['votes'] = entry.find(class_ = 'score unvoted')['title']
        post['comment_count'] = entry.find(class_ = 'comments').text.split()[0]
        post['comment_link'] = entry.find(class_ = 'comments')['href']
        post['time'] = entry.find(class_ = 'live-timestamp')['datetime']
        post['flair'] = ','. join([n.text for n in entry.find_all(class_ = "linkflairlabel")])
        raw_html_comm = requests.get(post['comment_link'], headers=headers)
        post['most_pop_comm'] = BeautifulSoup(raw_html_comm.content, "html.parser").find_all('form', class_ = 'usertext')[1].text
        frontpage_posts.append(post)
    except:
        pass

In [4]:
right_now = datetime.datetime.now()
now = right_now.strftime("%Y-%m-%d_%H_%M_%S")

In [5]:
df = pd.DataFrame(frontpage_posts)
df.to_csv('./reddit/reddit-frontpage_'+now+'.csv', index = False)